# Spotify Cold Start Models
Seth Billiau, William Drew, Sarah Lucioni


## Evaluating Models 

In [1]:
#RUN THIS CELL 
%run Models_module.ipynb

First, we need to initiate access to the Spotify API. We need to set our API keys and set up authentication with the API.

Let's now load the song data that we will use to train our models.

In [2]:
songs_test_df = pd.read_pickle("william/pickles/songs_30.pkl")
songs_test_df.head()


,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.00886,https://api.spotify.com/v1/audio-analysis/3Th5...,0.540,237147,0.557,3Th56VIq2sEaEmPPETu7p5,0.000182,11,0.3070,-6.097,1,0.0259,75.035,4,https://api.spotify.com/v1/tracks/3Th56VIq2sEa...,audio_features,spotify:track:3Th56VIq2sEaEmPPETu7p5,0.287
1,0.02320,https://api.spotify.com/v1/audio-analysis/6PCU...,0.461,261154,0.834,6PCUP3dWmTjcTtXY02oFdT,0.000011,2,0.1400,-4.868,1,0.0989,135.007,4,https://api.spotify.com/v1/tracks/6PCUP3dWmTjc...,audio_features,spotify:track:6PCUP3dWmTjcTtXY02oFdT,0.471
2,0.16300,https://api.spotify.com/v1/audio-analysis/0tgV...,0.599,263400,0.448,0tgVpDi06FyKpA1z0VMD4v,0.000000,8,0.1060,-6.312,1,0.0232,95.050,3,https://api.spotify.com/v1/tracks/0tgVpDi06FyK...,audio_features,spotify:track:0tgVpDi06FyKpA1z0VMD4v,0.168
3,0.58100,https://api.spotify.com/v1/audio-analysis/7qiZ...,0.825,233713,0.652,7qiZfU4dY1lWllzX7mPBI3,0.000000,1,0.0931,-3.183,0,0.0802,95.977,4,https://api.spotify.com/v1/tracks/7qiZfU4dY1lW...,audio_features,spotify:track:7qiZfU4dY1lWllzX7mPBI3,0.931
4,0.35500,https://api.spotify.com/v1/audio-analysis/51Ch...,0.761,238440,0.386,51ChrwmUPDJvedPQnIU8Ls,0.000000,4,0.0953,-6.158,1,0.0399,134.943,3,https://api.spotify.com/v1/tracks/51ChrwmUPDJv...,audio_features,spotify:track:51ChrwmUPDJvedPQnIU8Ls,0.526


## k-NN Models

SSE Method Evaluation

Functionalize

In [7]:
def evaluator(seed_song_uri, seed_song_name, length=49, k=50):
    df_raw_kmeans = run_all_kmeans(k, songs_df, seed_song_uri, length)
    df_raw_knn = run_all_knn(songs_df, seed_song_uri, length) 
    df_raw = pd.concat([df_raw_knn.iloc[:4],df_raw_kmeans], axis=0)
    df = df_raw.drop(labels='tempo', axis=1)
    residuals = df.iloc[:6] - df.iloc[6]
    sq_resid = np.square(residuals)
    sse = sq_resid.sum(axis = 1)
    sse = sse.rename(seed_song_name)
    return sse.to_frame()

Test function


In [147]:
series0 = evaluator("0TK2YIli7K1leLovkQiNik", "Senorita")
series1 = evaluator("61dYvvfIRtIDFuqZypPAta", "Yo-Yo Ma Cello Suite No. 1 in G Major")

/Users/sethbilliau/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


retrying ...1secs
retrying ...1secs


In [148]:
df = pd.concat([series0,series1], axis=1)

In [18]:
df = pd.read_csv("evals.csv")
df = df.set_index("Model")
df

,Senorita,Yo-Yo Ma Cello Suite No. 1 in G Major,Suge,bad guy,SICKO MODE,Despacito,Cyanide,Buy U a Drank (ShawtySnappin'),My Type,DDU-DU DDU-DU,King of Kings,Autumn Leaves,...,FACE,Inside Out,Lovebug,My Shot,Moments Passed,Come Down,Emaline,Say It Ain't So,Der Geist hilft unser Schwachheit auf,Vossi Bop,The Thrill is Gone,How Far I'll Go
Model,,,,,,,,,,,,,,,,,,,,,,,,,
k-NN/euclid/tempo,0.004108,0.007471,0.055340,0.045853,0.016450,0.009528,0.078642,0.026162,0.002327,0.000560,0.004246,0.087358,...,0.020528,0.028865,0.005819,0.004700,0.004245,0.028447,0.010707,0.010820,0.039614,0.000341,0.002741,0.000447
k-NN/euclid/no tempo,0.000153,0.000957,0.000941,0.011441,0.000360,0.000553,0.000545,0.001649,0.000053,0.000362,0.000084,0.002616,...,0.005342,0.000131,0.000600,0.000320,0.000423,0.001110,0.000139,0.000138,0.001006,0.029178,0.029912,0.006570
k-NN/sim/tempo,0.000804,0.008852,0.010914,0.015858,0.011922,0.009744,0.005271,0.130888,0.000194,0.012985,0.006912,0.026214,...,0.051708,0.008655,0.059192,0.000535,0.028958,0.020129,0.000105,0.000593,0.004548,0.001237,0.058425,0.005086
k-NN/sim/no tempo,0.000218,0.006980,0.018576,0.014628,0.006341,0.003939,0.005738,0.078595,0.001166,0.004150,0.046792,0.002551,...,0.083133,0.002886,0.028596,0.000413,0.015484,0.058544,0.000044,0.022434,0.005444,0.006090,0.055224,0.011233
k-means/k=50/tempo,0.163915,1.112812,0.332391,0.139678,0.278779,0.107131,0.236618,0.214501,0.169776,0.250433,0.116599,0.267342,...,0.272206,0.218657,0.103388,0.179155,0.132276,0.180081,0.237566,0.202545,0.698867,0.119955,0.147720,0.219445
k-means/k=50/no tempo,0.134497,1.121155,0.391957,0.144949,0.157548,0.063538,0.263430,0.177378,0.160391,0.299767,0.100181,0.289967,...,0.239928,0.239411,0.116032,0.239149,0.145516,0.157483,0.137893,0.169659,0.637459,0.166315,0.132937,0.334911


In [27]:
songs_test = [
#              ("2gwkD6igEhQbDQegRCcdoB", "Suge"), 
#              ("2Fxmhks0bxGSBdJ92vM42m", "bad guy"),
#              ("2xLMifQCjDGFmkHkpNLD9h", "SICKO MODE"),
#              ("6habFhsOp2NvshLv26DqMb", "Despacito"),
#              ("3uouaAVXpQR3X8RYkJyitQ", "Cyanide"), 
#              ("2aIdVb8v9KTpEZnftkz2mD", "Buy U a Drank (ShawtySnappin')"),
#              ("2Iq6HhIquO7JKr0KfTNLzU", "My Type"),
#              ("2aI2k39nfa3KFsa4JclQzw", "DDU-DU DDU-DU"),
#              #("3aW0ds4A4tSQDIp75FqWTo","FAKE LOVE"),
#              ("6kmhapLx51lmwhIIDFsUQU", "King of Kings"), 
#              ("1QlTNF43OyKK9BorIRsJ8o", "Autumn Leaves"),
#              ("3LpheVDaZXkUePlKbZi9Md", "The Star Wars Theme"), 
#              ("23if4cvw0UI7c8Uc5OOvss", "The Marriage of Figaro Overture"),
#              ("0bYg9bo50gSsH3LtXe2SQn", "All I Want for Christmas is You"),
#              ("5ta5qajKBkZcV7KR6laOTt", "Can't Take my Eyes off of You"),
#              ("2KBQH1GGhLzS4N0vNS1HJF", "No Woman, No Cry"),
#              ("5MJfH9FmMur9gQPqIpXLwP", "William Shatner speaks Rocket Man"),
#              ("2tJulUYLDKOg9XrtVkMgcJ", "Grenade"),
#              ("0yD66650JxhqKbW76C2qCo", "Country Girl (Shake It For Me)"),
#              ("1kBbgxAUc4HQv7cgIayIDH", "Ransom"),
#              ("1e1JKLEDKP7hEQzJfNAgPl", "Magnolia"), 
#              ("5bknBRjKJZ643DAN2w8Yoy", "FACE"),
#              ("4Htt3QaBWdLggq88rJI5MU", "Inside Out"),
#              ("3F8xLc6T4VvKH7I6wqVAN2", "Lovebug"),
#              ("4cxvludVmQxryrnx1m9FqL", "My Shot"),
#              ("2iE8Ymbdcm9AEniaxSFcbL", "Moments Passed"),
#              ("276zciJ7Fg7Jk6Ta6QuLkp", "Come Down"),
#              ("3bZZyVuolDKTLgS9gSQZLr", "Emaline"),
#              ("6VoIBz0VhCyz7OdEoRYDiA", "Say It Ain't So"),
#              ("4ZXLWTmQFzM02hZwMiZfgS", "Der Geist hilft unser Schwachheit auf"),
#              ("44ZJ4hLD5onikKsVLQaILr", "Vossi Bop"),
#              ("4VCl2a5JvdrEx6PL2P29ty", "The Thrill is Gone"),
#              ("6mb6lVLNrcUgLnEN8QnDJd", "How Far I'll Go"),
#              ("3yOlyBJuViE2YSGn3nVE1K", "My Oh My"),
#              ("3rQ0ZaLrkLv8HhAAKSbAC0", "Posthumous Forgiveness"),
#              ("2TjnCxxQRYn56Ye8gkUKiW", "Desperado"),
#              ("4pNiE4LCVV74vfIBaUHm1b", "Sir Duke"),
#              ("04WQx5XWBaExK6vEP0gaO0", "Happy Birthday"),
#              ("4wnpMSadXzCtHa0mvK2PD4", "Star Spangled Banner"),
#              ("5OCJzvD7sykQEKHH7qAC3C", "God is a woman"),
#              ("7IkEC7xL7C1RIoqOsOlkZb", "God Save the Queen (Queen)"),
#              ("4YpHCvwccTpV2M94Fx2zc2", "O Canada"),
#              ("2Kz5yAW5qdVVpKHR5gVESi", "Kyrie (Mozart Requiem)"),
#              ("0rKtyWc8bvkriBthvHKY8d", "SLOW DANCING IN THE DARK"),
#              ("3GRSqlALWISqLeNncZMbpX", "Mean It"),
#              ("09H7Z0qX0KgYCIoPgqY9YF", "A Sea Symphony, Movt 2"),
#              ("2LSsFt8nWQrm6mTlAMawOs", "Thais Meditation"),
#              ("3kdmTAhpLsv3scnzE90Fue", "Bogoroditse Devo"),

# ADD NEW SONGS
#              ("3kdmTAhpLsv3scnzE90Fue", "Bogoroditse Devo"),
#              ("3kdmTAhpLsv3scnzE90Fue", "Bogoroditse Devo"),
             ]
len(songs_test)


2

In [32]:
for song in songs_test: 
    series = evaluator(song[0], song[1])
    df = pd.concat([df,series], axis=1)
    print(song[1])

Thais Meditation
Bogoroditse Devo


In [37]:
df_partial = df.copy()
html = df_partial.transpose()
html.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Model</th>\n      <th>k-NN/euclid/tempo</th>\n      <th>k-NN/euclid/no tempo</th>\n      <th>k-NN/sim/tempo</th>\n      <th>k-NN/sim/no tempo</th>\n      <th>k-means/k=50/tempo</th>\n      <th>k-means/k=50/no tempo</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Senorita</th>\n      <td>0.004108</td>\n      <td>0.000153</td>\n      <td>0.000804</td>\n      <td>0.000218</td>\n      <td>0.163915</td>\n      <td>0.134497</td>\n    </tr>\n    <tr>\n      <th>Yo-Yo Ma Cello Suite No. 1 in G Major</th>\n      <td>0.007471</td>\n      <td>0.000957</td>\n      <td>0.008852</td>\n      <td>0.006980</td>\n      <td>1.112812</td>\n      <td>1.121155</td>\n    </tr>\n    <tr>\n      <th>Suge</th>\n      <td>0.055340</td>\n      <td>0.000941</td>\n      <td>0.010914</td>\n      <td>0.018576</td>\n      <td>0.332391</td>\n      <td>0.391957</td>\n    </tr>\n    <tr>\n      <th>bad guy</th>\n 

In [34]:
df_partial.to_csv("evals.csv")

In [35]:
df.mean(axis=1).sort_values()

Model
k-NN/euclid/no tempo     0.002360
k-NN/sim/tempo           0.013097
k-NN/sim/no tempo        0.014297
k-NN/euclid/tempo        0.022173
k-means/k=50/tempo       0.382007
k-means/k=50/no tempo    0.393498
dtype: float64